In [1]:
from pprint import pprint

# def run_sql_command():
#     session = SessionFactory()
#     try:
#         session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
#         session.commit()
#     finally:
#         session.close()

In [2]:
# Open a connection to the database

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "postgresql+psycopg2://user:password@db/vectordb"

engine = create_engine(DATABASE_URL)
SessionFactory = sessionmaker(autocommit=False, autoflush=False, bind=engine)

session = SessionFactory()

In [3]:
# Create a table with a vector column

from sqlalchemy import Integer, String
from sqlalchemy.orm import mapped_column, declarative_base
from pgvector.sqlalchemy import Vector

Base = declarative_base()

class Item(Base):
    __tablename__ = 'embeddings'
    id = mapped_column(Integer, primary_key=True)
    embedding = mapped_column(Vector(768))
    text = mapped_column(String)

    def __repr__(self):
        return f"<Item(text={self.text[:30]}...)>"

In [4]:
# Create example documents for embedding

from ollama import Client

client = Client(host='ollama:11434')

documentation = [
    {
        "title": "API Documentation - User Service",
        "content": "The User Service API allows CRUD operations on user data. The base URL is '/api/users'. GET /api/users retrieves all users. POST /api/users creates a new user."
    },
    {
        "title": "Architecture Guidelines - Microservices",
        "content": "Our architecture is based on microservices. Each service is independent and communicates with others via REST APIs. Services are deployed in Docker containers and managed via Kubernetes."
    },
    {
        "title": "Code Standards",
        "content": "Our codebase follows PSR-12 for PHP code. All PHP classes should have docblocks, and methods should be named in camelCase. JavaScript follows the Airbnb style guide."
    },
    {
        "title": "CI/CD Pipeline",
        "content": "We use GitLab CI for continuous integration and deployment. The pipeline includes stages for linting, testing, building, and deployment. Automated tests are required before deployment."
    }
]

emb_data = []
for doc in documentation:
    vector = client.embeddings(model='nomic-embed-text', prompt=doc['content'])
    embedding = vector['embedding']
    content = doc['content']
    emb_data.append((embedding, content))

In [5]:
# Write vector data into the database

for embedding, content in emb_data:
    item = Item(embedding=embedding, text=content)
    session.add(item)
    
session.commit()
session.close()

In [6]:
# Create a question (embeddings)

question = "How do we deploy services in our architecture?"
question_vector = client.embeddings(model='nomic-embed-text', prompt=question)['embedding']

In [12]:
# Query the database for similar documents

results = (
    session.query(
        Item,
        (Item.embedding.l2_distance(question_vector)).label('distance')
    ).order_by('distance').limit(5).all()
)

# for item, distance in results:
#     pprint.pp(item.text)
#     pprint.pp(distance)

('Our architecture is based on microservices. Each service is independent and '
 'communicates with others via REST APIs. Services are deployed in Docker '
 'containers and managed via Kubernetes.')


In [15]:
# Based on the vector search (similarity) results, ask llama to generate a response.
# For better implementation check this: https://python.langchain.com/docs/integrations/chat/llama2_chat/

from ollama import ChatResponse

internal_question = f"""
Based on the following retrieved documents:
---
1. {results[0][0].text}
---
2. {results[1][0].text}
---
3. {results[2][0].text}
---
Answer the question: "{question}"
"""

response: ChatResponse = client.chat(model='llama3.2', messages=[
    {
        'role': 'user',
        'content': internal_question,
    },
])

pprint(response['message']['content'])

# or access fields directly from the response object
# print(response.message.content)

('According to the retrieved documents, services are deployed in Docker '
 'containers and managed via Kubernetes.')
